In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

from poke_env.player_configuration import PlayerConfiguration
from poke_env.player.random_player import RandomPlayer
from poke_env.player.baselines import SimpleHeuristicsPlayer
from poke_env.server_configuration import ShowdownServerConfiguration
from poke_env.player.utils import cross_evaluate

from tabulate import tabulate

%load_ext autoreload
%autoreload 2

In [4]:
player = SimpleHeuristicsPlayer(
    player_configuration=PlayerConfiguration("JoeNextLine", "underground"),
    server_configuration=ShowdownServerConfiguration,
)

await player.send_challenges("meatout", n_challenges=1)


In [2]:
import json
from dotmap import DotMap 


from pokebot.bots.state_engine import SimpleStateEngine
from pokebot.bots.bot import BotPlayer
from pokebot.models.dqn import Model
from pokebot.trainers.trainer import SimpleDQNTrainer

C:\Users\nacha\Miniconda3\envs\pk_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nacha\Miniconda3\envs\pk_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nacha\Miniconda3\envs\pk_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nacha\Miniconda3\envs\pk_env\lib\site-packages\tensorflow\

In [3]:
hparams = DotMap(json.load(open('./hparams.json', 'r')))
p_dict = hparams.policy
a_dict = hparams.agent

print((p_dict))

DotMap(attr='eps', value_max=1.0, value_min=0.5, value_test=0, nb_steps=10000)


In [4]:
player_configuration=PlayerConfiguration("JoeNextLine", "underground"),
server_configuration=ShowdownServerConfiguration,

player = BotPlayer(
    player_configuration=PlayerConfiguration("test1",None),
#     log_level=10,
    state_engine=SimpleStateEngine(10)
    )

model = Model(player)

In [5]:
trainer = SimpleDQNTrainer(player, model, p_dict, a_dict)

In [6]:
await trainer.train()

Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: 0.3983
210 episodes - episode_reward: 18.981 [-44.130, 47.322] - loss: 0.011 - mae: 0.353 - mean_q: 0.572 - mean_eps: 0.725

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 105s 11ms/step - reward: 0.6677
246 episodes - episode_reward: 27.129 [-41.040, 47.801] - loss: 0.014 - mae: 0.322 - mean_q: 0.509 - mean_eps: 0.500

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 105s 11ms/step - reward: 0.7298
259 episodes - episode_reward: 28.147 [-45.200, 47.165] - loss: 0.014 - mae: 0.323 - mean_q: 0.507 - mean_eps: 0.500

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 105s 11ms/step - reward: 0.6626
249 episodes - episode_reward: 26.631 [-41.010, 47.143] - loss: 0.014 - mae: 0.337 - mean_q: 0.530 - mean_eps: 0.500

Interval 5 (40000 steps performed)
10000/10000

In [8]:
from poke_env.player.baselines import MaxBasePowerPlayer, SimpleHeuristicsPlayer

opponents = [MaxBasePowerPlayer(battle_format="gen8randombattle"),
             SimpleHeuristicsPlayer(battle_format="gen8randombattle")]

await trainer.evaluate(opponents)

Results against player: <poke_env.player.baselines.MaxBasePowerPlayer object at 0x00000234250CF2E8>
DQN Evaluation: 69 victories out of 100 episodes
Results against player: <poke_env.player.baselines.SimpleHeuristicsPlayer object at 0x00000234250B8748>
DQN Evaluation: 15 victories out of 100 episodes


In [11]:
#Save current player

trainer.agent.save_weights("../data/agents/simple_dqn.hdf5")

In [ ]:
#Load agent

